In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)

        self.fc1 = nn.Linear(in_features=16 * 5 * 5, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.out = nn.Linear(in_features=84, out_features=10)

    def forward(self, t):
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        t = t.reshape(-1, 16 * 5 * 5)
        t = self.fc1(t)
        t = F.relu(t)

        t = self.fc2(t)
        t = F.relu(t)

        t = self.out(t)
        return t


# грузим лучшую обученную модель
model = LeNet().to(device)
state = torch.load("lenet_cifar10_best_pytorch.pth", map_location=device)
model.load_state_dict(state)
model.eval()

# фиктивный вход для трассировки
dummy_input = torch.randn(200, 3, 32, 32, device=device)

# экспорт в ONNX
onnx_program = torch.onnx.export(
    model,
    dummy_input,
    input_names=["input"],
    output_names=["logits"],
)


print("Экспортирован в lenet_cifar10.onnx")


Device: cuda
[torch.onnx] Obtain model graph for `LeNet([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `LeNet([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Экспортирован в lenet_cifar10.onnx


In [2]:
onnx_program.save("lenet_cifar10.onnx")

In [4]:
import onnx

onnx_model = onnx.load("lenet_cifar10.onnx")
onnx.checker.check_model(onnx_model)